In [61]:
#importing required libraries
import numpy as np
from sklearn.mixture import GaussianMixture
from sklearn.datasets import load_iris
from scipy.spatial.distance import euclidean
from sklearn.metrics import confusion_matrix



In [62]:
#uploading the iris data directly from the UCI machine learning repository
from ucimlrepo import fetch_ucirepo

# fetch dataset 
iris = fetch_ucirepo(id=53) 
  
# data (as pandas dataframes) 
data = iris.data.features 
target = iris.data.targets 
  
# metadata 
print(iris.metadata)

# variable information 
print(iris.variables) 

{'uci_id': 53, 'name': 'Iris', 'repository_url': 'https://archive.ics.uci.edu/dataset/53/iris', 'data_url': 'https://archive.ics.uci.edu/static/public/53/data.csv', 'abstract': 'A small classic dataset from Fisher, 1936. One of the earliest known datasets used for evaluating classification methods.\n', 'area': 'Life Science', 'tasks': ['Classification'], 'characteristics': ['Tabular'], 'num_instances': 150, 'num_features': 4, 'feature_types': ['Real'], 'demographics': [], 'target_col': ['class'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 1936, 'last_updated': 'Tue Sep 12 2023', 'dataset_doi': '10.24432/C56C76', 'creators': ['R. A. Fisher'], 'intro_paper': {'title': 'The Iris data set: In search of the source of virginica', 'authors': 'A. Unwin, K. Kleinman', 'published_in': 'Significance, 2021', 'year': 2021, 'url': 'https://www.semanticscholar.org/paper/4599862ea877863669a6a8e63a3c707a787d5d7e', 'doi': '1740-9713.01589'},

In [63]:
X = data 

#clusters
k = 3

#points per cluster = n/k
cluster_len = len(X) // k

#storing the mean for each cluster in a list
initial_means = []

In [64]:
# Expected maximization
for i in range(k):
    s_i = i * cluster_len
    e_i = (i + 1) * cluster_len
    cluster_data = X[s_i:e_i]
    #calculating the new mean and appending it
    mean = np.mean(cluster_data, axis=0)
    initial_means.append(mean) 

In [65]:
#guassian mixture calculation 
gmm = GaussianMixture(n_components=n_clusters, means_init=np.array(initial_means),
                      covariance_type='full')

# intializing the maximum number of iterations,convergence tolerance
max_iterations = 50
tolerance = 0.000001  
converged = False

#checking for convergence
for i in range(max_iterations):
    gmm.fit(X)
    if i > 0:
        mean_distances = [euclidean(old_mean, new_mean) for old_mean, new_mean in zip(old_means, gmm.means_)]
        if all(distance < tolerance for distance in mean_distances):
            converged = True
            break
    old_means = gmm.means_.copy()

In [68]:
if converged:
    print(f"Converged after {i+1} iterations.\n")
else:
    print("Did not converge within the maximum number of iterations.\n")   


Converged after 3 iterations.



In [69]:
sorted_means = sorted(zip(initial_means, gmm.means_), key=lambda x: np.linalg.norm(x[0]))

for i, (initial_mean, final_mean) in enumerate(sorted_means):
    print(f"Mean {i+1} (Norm: {np.linalg.norm(final_mean):.6f}):\n{final_mean}\n")

Mean 1 (Norm: 6.240640):
[5.006 3.418 1.464 0.244]

Mean 2 (Norm: 7.871425):
[5.91173838 2.77894708 4.19651417 1.29514342]

Mean 3 (Norm: 9.236381):
[6.54088231 2.9460898  5.47090074 1.97924237]



In [70]:
#covariance matrix of each cluster 
for i, (initial_mean, final_mean) in enumerate(sorted_means):
    covariance_matrix = gmm.covariances_[mean_index]
    print(f"Covariance Matrix for Mean {i+1}:\n{covariance_matrix}\n")

Covariance Matrix for Mean 1:
[[0.38656354 0.09273542 0.3043059  0.0634571 ]
 [0.09273542 0.11084912 0.08645572 0.05724244]
 [0.3043059  0.08645572 0.33350613 0.07892995]
 [0.0634571  0.05724244 0.07892995 0.08814876]]

Covariance Matrix for Mean 2:
[[0.38656354 0.09273542 0.3043059  0.0634571 ]
 [0.09273542 0.11084912 0.08645572 0.05724244]
 [0.3043059  0.08645572 0.33350613 0.07892995]
 [0.0634571  0.05724244 0.07892995 0.08814876]]

Covariance Matrix for Mean 3:
[[0.38656354 0.09273542 0.3043059  0.0634571 ]
 [0.09273542 0.11084912 0.08645572 0.05724244]
 [0.3043059  0.08645572 0.33350613 0.07892995]
 [0.0634571  0.05724244 0.07892995 0.08814876]]



In [88]:
#cluster assignments for each data point
cluster_assignments = gmm.predict(X)


for i in range(n_clusters):
    members = np.where(cluster_assignments == i)[0]  
    sorted_members = np.sort(members)  
    print(f"Cluster {i+1}: {', '.join(map(str, sorted_members))}")

# final size of each cluster
cluster_sizes = [np.sum(cluster_assignments == i) for i in range(n_clusters)]
print("Size:", ' '.join(map(str, cluster_sizes)))

Cluster 1: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49
Cluster 2: 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 69, 71, 73, 74, 75, 76, 78, 79, 80, 81, 82, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99
Cluster 3: 68, 70, 72, 77, 83, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149
Size: 50 45 55


C:\Users\13179\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but GaussianMixture was fitted with feature names
  warnings.warn(


In [89]:
# Load the Iris dataset
iris = load_iris()
X = iris.data  # Features
y_true = target  # True labels


In [90]:
# Load the Iris dataset
iris = load_iris()
X = iris.data  # Features
y_true = iris.target  # True labels

# Define the number of clusters (k)
n_clusters = 3


# Compute cluster assignments for each data point
cluster_assignments = gmm.predict(X)

# Compute confusion matrix to determine purity
conf_matrix = confusion_matrix(y_true, cluster_assignments)

# Calculate purity
purity = np.sum(np.amax(conf_matrix, axis=0)) / len(X)

# Display purity score
print(f"Purity: {purity:.6f}")

Purity: 0.966667


C:\Users\13179\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but GaussianMixture was fitted with feature names
  warnings.warn(
